---
title: "14-SAS 卡方检验2-四格表资料卡方检验"
author: "Simon Zhou"
date: "2025-06-06"
date-modified: "2025-06-06"
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [2]:
%load_ext saspy.sas_magic

## RxC表资料的$\chi^2$检验

行x列表资料根据行变量和列变量的类型,可以分成3种类型:双向无序列联表资料、单向有序列联表资料和双向有序列联表资料。

对于这三种类型的资料可用 SAS 提供的 CMH 统计量( Cochran-Mantel-Haenszel Statisitic)进行分析,其包括3个统计量即:

1. Nonzero Correlation:行变量和列变量为非零相关,可用于双向有序的资料;
2. Row Mean Scores Difer:行均数得分差值,可用于列变量为有序变量的资料;
3. General Association:行、列变量为一般关联,可用于双向无序的资料。

## 双向无序资料

双向无序资料的行变量和列变量都是名义变量,如职业、血型、疾病的类型等,这种变量的各水平间无内在的有序关联。分析这种资料,目的在于检验两变量的关系是否独立。

### 示例

以下是提取后的 Markdown 表格：

|ABO 血型|MN 血型-M|MN 血型-N|MN 血型-MN|合计|
| ---- | ---- | ---- | ---- | ---- |
|O| 431|490|902|1823|
|A| 388|410|800|1598|
|B| 495|587|950|2032|
|AB| 137|179|32|348|
|合计| 1451|1666|2684|5801|

In [ ]:
%%SAS
/*程序14-1*/
data chi14_1;
    input r c f @@;
datalines;
1 1 431 1 2 490 1 3 902
2 1 388 2 2 410 2 3 800
3 1 495 3 2 587 3 3 950
4 1 137 4 2 179 4 3 32
;
proe freq;
    weight f;
    tables r*c/cmh norow nocol nopercent;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 5264



## 单向有序资料

单向有序资料表示行变量是名义变量,列变量是有序变量,如疗效分为治愈、显效、好转、无效等。

SAS对列变量为有序变量的资料,将列变量的各水平依次进行评分,再比较行变量的各水平间的平均得分的差别是否有统计学意义。

对行变量为有序变量的资料则按双向有序资料进行处理。

### 示例


|地区|轻度（1）|中度（2）|较重度（3）|严重（4）|合计|
| ---- | ---- | ---- | ---- | ---- | ---- |
|城市（1）|2211|949|296|71|3527|
|农村（2）|670|330|115|52|1167|
|合计|2881|1279|411|123|4694|

In [3]:
%%SAS
/*程序14-2*/
data chi14_2;
    do r =1 to 2;
        do c =1 to 4;
            input f @@;
            output;
        end;
    end;
datalines;
2211 949 296 71 670 330 115 52
;
proe freq;
    tables r*c/cmh nopercent nocol;
    weight f;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 37228



### 结果说明

选择 `2	行评分均值差异	1	20.3626	<.0001` 作为此处单向有序资料卡方检验的结果，说明差异具有统计学意义。

## 双向有序资料

双向有序资料表示行变量和列变量都是有序变量,SAS在处理这种资料时,对行变量和列变量分别依次进行评分,然后检验这两个变量之间是否有相关关系。

### 示例


|年龄\等级| - | + | ++ | +++ | 合计 |
| ---- | ---- | ---- | ---- | ---- | ---- |
|20~| 70|22|4|2|98|
|30~| 27|24|9|3|63|
|40~| 16|23|13|7|59|
|≥50| 9|20|15|14|58|
|合计| 122|89|41|26|278|

In [7]:
%%SAS
/*程序14-3*/
data chi14_3;
    do r = 1 to 4;
        do c =1 to 4;
            input f@@;
            output;
        end;
    end;
datalines;
70 22 4 2 27 24 9 3 16 23 13 7 9 20 15 14
;
proc freq;
    weight f;
    tables r*c/cmh nopercent norow nocol;
run;

### 结果说明

这里是双向有序资料，所以选用 `1	非零相关	1	63.3895	<.0001` 作为这里的结果，$\chi^2_{CMH}=63.3895,P<0.0001$，差异有统计学意义，说明年龄和严重等级有相关关系。

## 分层资料

CMH 统计量也可用于多层行x列表资料的卡方检验,即按一个或多个因素分层后,研究行变量和列变量间的联系。可通过控制分层变量的影响后,检验研究行变量和列变量的关系。

### 示例


|性别|组别|使用(c=1)|未使用(c=2)|
| ---- | ---- | ---- | ---- |
|男(sex=1)|病例组(r=1)|5|36|
|男|对照组(r=2)|33|645|
|女(sex=2)|病例组(r=1)|10|58|
|女|对照组(r=2)|19|518|

In [9]:
%%SAS
/*程序14-4*/
data chi14_4;
    do c=1 to 2;
        do sex=1 to 2;
            do r=1 to 2;
                input f@@;
                output;
            end;
        end;
    end;
datalines;
5 33 10 19 36 645 58 518
;
proc freq;
    weight f;
    tables sex*r*c/cmh nopercent norow nocol;
run;

### 结果说明

整个结果共分三个部分：

1. 第一部分输出的是输出两张列联表,分别是 `sex=1` 和 `sex=2` 时的二维列联表。
2. 第二部分输出在控制分层变量sex后总的检验 Cochran-Mantel-Haenszel的统计结果,由于行变量和列变量都只有两个,可以看作双向无序资料,三种结果是一致的,结果为: $\chi^2_{CHM}=19.5130$ ,所对应的 P<0.0001,说明控制了性别因素后,使用别嘌呤醇与发生药物性皮疹是有关系的。
3. 第三部分是一些相对数统计量的比值,第一个是病例对照(优比),即病例-对照研究中的优势比(又称比值比,OR),有两个统计结果,个是用 Mantel-Haenszel 方法计算的值,另一个是用 Logit 方法计算的值,在病例-对照研究中,优势比是非常重要的指标,本例说明使用别嘌呤醇发生药物性皮疹的危险性是不使用该药危险性的4倍(3.7560或3.7775)。后面两个是 OR的 95%置信区间,本例为(2.015 8,6.9983)或(2.011 4,7.094 1),其结论与 CMH 结论相同。下面两个都是队列研究所用的统计指标(相对危险度,RR),区别在于分别将第1列或第2列作为发病情况。
4. 第四部分为对优势比进行 Breslow-Day 齐性检验,本例 $\chi^2=0.7029,P=0.401 8>0.05$,说明不同分层之间的优势比具有齐性，可认为它们一致。

## `PROC FREQ` 过程常用选项与语句详解

### `PROC FREQ` 过程的基本格式

```sas
proc freq <选项>;
    by 变量名1 <变量名2>...;
    tables 表格定义项 <选项>;
    output out=输出数据集 <关键词>;
    test <关键词>;
    weight 权重变量;
run;
```

#### 各语句说明：

* `by`：用于对分组变量进行分层分析，需先排序。
* `tables`：用于指定交叉表分析的变量及表格结构，是最常用的语句。
* `output`：将分析结果输出为数据集，便于后续处理。
* `test`：用于指定统计检验，如卡方检验、Fisher精确检验等。
* `weight`：指定频数变量，适用于加权样本数据。

### `TABLES` 语句常用选项详解

`TABLES` 是 `PROC FREQ` 中的核心语句之一，除了基础的交叉表功能，还可通过附加选项实现更复杂的统计分析：

#### `riskdiff`

用于二分类四格表（2×2表）中，计算**两组间的事件发生率差异**（即风险差 Risk Difference, RD）及其置信区间。

提供的指标包括：

* 每行的事件发生率（即比例或风险）
* 风险差（两行比例之差）
* 渐近标准误（依据正态分布近似）
* 渐近 95% 置信区间
* 精确 95% 置信区间（基于二项分布）

**应用场景**：队列研究、临床试验中对比暴露组与对照组的疾病发生率。

```sas
tables exposure*outcome / riskdiff;
```

#### `relrisk`

用于四格表中，计算**相对风险（Relative Risk, RR）**或**比值比（Odds Ratio, OR）**，及其置信区间。

* 若数据源自**队列研究**：计算 RR（相对风险）
* 若数据源自**病例对照研究**：计算 OR（比值比）

返回值包括：

* RR 或 OR 的点估计
* 渐近 95% 置信区间

**使用示例**：

```sas
tables exposure*disease / relrisk;
```

### 其他常用选项：

* `chisq`：卡方检验（包括Pearson卡方、Yates校正和Fisher精确检验）
* `nocol` / `norow` / `nopercent`：抑制列百分比、行百分比、总百分比的显示
* `expected`：显示期望频数（用于判断适用卡方检验的前提）
* `measures`：输出列联表的各种关联性度量指标，如 Phi、Cramer’s V、Contingency Coefficient 等


### 补充说明：`OUTPUT` 与 `TEST` 语句

#### `output` 语句常用关键字：

* `out=`：指定输出结果数据集
* `n nlevel`：输出各变量频数和水平数
* `pct`：输出比例
* `chisq`：输出卡方值
* `relrisk`：输出相对风险及置信区间

例如：

```sas
output out=result_freq chisq relrisk;
```

#### `test` 语句常用于非参数检验，如：

```sas
test chisq;
```

---

### 示例代码

```sas
proc freq data=clinic;
    tables treatment*response / chisq riskdiff relrisk expected;
    output out=freq_results chisq relrisk;
run;
```

---

### 适用场景总结

| 场景              | 推荐选项           |
| --------------- | -------------- |
| 检验两个分类变量是否独立    | `chisq`        |
| 计算列联表中比例差异（风险差） | `riskdiff`     |
| 计算 RR/OR 及其区间   | `relrisk`      |
| 临床或流行病学数据分层分析   | `by`, `tables` |
